In [ ]:
!curl -O https://s3.amazonaws.com/text-datasets/nietzsche.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  586k  100  586k    0     0  2596k      0 --:--:-- --:--:-- --:--:-- 2596k


In [ ]:
import tensorflow as tf 
import numpy as np 
input_sequence_size = 4
max_features = 54762
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE
def convert_string(X: tf.Tensor):
  str1 = ""
  for elem in X:
    str1 += elem.numpy().decode("utf-8") + ""
  str1 = tf.convert_to_tensor(str1[:-1])
  return str1
def _fixup_shape(x, y):
  x.set_shape([4])
  y.set_shape([1])
  return x,y
raw_data_ds = tf.data.TextLineDataset(['nietzsche.txt'])
raw_data_ds = raw_data_ds.map(lambda x: tf.strings.split(x))
raw_data_ds = raw_data_ds.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))
sequence_data_ds = raw_data_ds.window(input_sequence_size+1, drop_remainder=True)
sequence_data_ds = sequence_data_ds.flat_map(lambda x: x.batch(5))
sequence_data_ds = sequence_data_ds.map(lambda x: (x[:-1], x[-1:]))
X_train_ds = sequence_data_ds.map(lambda x,y: x)
Y_train_ds = sequence_data_ds.map(lambda x,y: y)
X_train_ds = X_train_ds.map(lambda x: tf.py_function(func = convert_string, inp = [x], Tout = tf.string))
X_train_ds = X_train_ds.map(lambda x: tf.reshape(x,[1]))
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens = max_features,
    output_mode = "int",
    output_sequence_length = input_sequence_size
)
vectorize_layer.adapt(X_train_ds)

In [ ]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))
X_train_ds = X_train_ds.map(vectorize_text)
Y_train_ds = Y_train_ds.map(vectorize_text)
Y_train_ds = Y_train_ds.map(lambda x: x[0])
train_ds = tf.data.Dataset.zip((X_train_ds, Y_train_ds))
train_ds = train_ds.map(_fixup_shape)
train_ds = train_ds.shuffle(buffer_size=512).batch(batch_size, drop_remainder=True).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds.take(1) 
train_ds.element_spec

(TensorSpec(shape=(128, 4), dtype=tf.int64, name=None),
 TensorSpec(shape=(128, 1), dtype=tf.int64, name=None))